In [ ]:
import numpy
import numpy as np

total_auc = {}
for split_part in np.arange(0.1, 1.0, 0.1):
    clf_ratio = np.round(split_part,2) 
    #save_file = 'Foursquare/split_data/'+str(clf_ratio)+'_train_file.txt'
    save_file = 'Gowalla/split_data/'+str(clf_ratio)+'_train_file.txt'
    text_id = {}
    id_text = {}
    
    i = 0
    def read_node_label(filename, i):
        fin = open(filename, 'r')
        while 1:
            l = fin.readline()
            if l == '':
                break
            vec = l.strip().split('\t')
            sid = vec[0]
            tid = vec[1]
            if sid not in text_id.keys():
                text_id[sid] = i
                i += 1
            if tid not in text_id.keys():
                text_id[tid] = i
                i += 1
        fin.close()

    read_node_label(save_file, i)
    #save_file = 'Foursquare/split_data/'+str(clf_ratio)+'_test_file.txt'
    save_file = 'Gowalla/split_data/'+str(clf_ratio)+'_test_file.txt'
    read_node_label(save_file, i)

    for i in range(len(text_id)):
        for j in text_id.keys():
            if text_id[j] == i:
                id_text[i] = j
                continue
                
    
##########    
    #save_file = 'Foursquare/split_data/'+str(clf_ratio)+'_test_file.txt'
    save_file = 'Gowalla/split_data/'+str(clf_ratio)+'_test_file.txt'
    fin = open(save_file, 'r')
    #filename = 'Foursquare/split_data/'+str(clf_ratio)+'_reorder_test.txt'
    filename = 'Gowalla/split_data/'+str(clf_ratio)+'_reorder_test.txt'
    fout = open(filename, 'w')
    
    while 1:
        l = fin.readline()
        if l == '':
            break
        vec = l.strip().split('\t') 
        uid = text_id[vec[0]]
        lid = text_id[vec[1]]
        fout.write("{}\t{}\n".format(uid,lid))
    fin.close()
    fout.close()
    
    #save_file = 'Foursquare/split_data/'+str(clf_ratio)+'_train_file.txt'
    save_file = 'Gowalla/split_data/'+str(clf_ratio)+'_train_file.txt'
    fin = open(save_file, 'r')
    #filename = 'Foursquare/split_data/'+str(clf_ratio)+'_reorder_train.txt'
    filename = 'Gowalla/split_data/'+str(clf_ratio)+'_reorder_train.txt'
    fout = open(filename, 'w')
    
    while 1:
        l = fin.readline()
        if l == '':
            break
        vec = l.strip().split('\t') 
        uid = text_id[vec[0]]
        lid = text_id[vec[1]]
        fout.write("{}\t{}\n".format(uid,lid))
    fin.close()
    fout.close()
##########    


    user_geo = {}
    loca_geo = {}
    X_train = []

    #label_file = 'Foursquare/split_data/'+str(clf_ratio)+'_train_file.txt'
    label_file = 'Gowalla/split_data/'+str(clf_ratio)+'_train_file.txt'
    
    def read_node_label(filename):
        fin = open(filename, 'r')
        while 1:
            l = fin.readline()
            if l == '':
                break
            vec = l.strip().split('\t')

            uid = vec[0] 
            lid = vec[1]
            geo = vec[2]

            if uid not in user_geo.keys():
                user_geo[uid] = geo

            if lid not in loca_geo.keys():
                loca_geo[uid] = geo

            tmp_geo = geo.split(',')

            X_train.append([float(tmp_geo[0]), float(tmp_geo[1])])
        fin.close()
    read_node_label(label_file)
        
    
    import numpy as np
    X_train = np.asarray(X_train)
    import matplotlib.pyplot as plt
    from matplotlib.colors import LogNorm
    from sklearn import mixture

    np.random.seed(0)
    clf = mixture.GaussianMixture(n_components=100, covariance_type='full')
    clf.fit(X_train)
    
    
    pre_X = {}

    #label_file = 'Foursquare/split_data/'+str(clf_ratio)+'_train_file.txt'
    label_file = 'Gowalla/split_data/'+str(clf_ratio)+'_train_file.txt'
    
    def read_node_label(filename):
        line = 0
        fin = open(filename, 'r')
        while 1:
            l = fin.readline()
            if l == '':
                break
            vec = l.strip().split('\t')

            uid = vec[0] 
            lid = vec[1]
            geo = vec[2].split(',')

            tmp_geo = []
            tmp_geo.append( [float(geo[0]), float(geo[1])] )
            tmp_geo = np.asarray(tmp_geo)

            log_prob = clf._estimate_log_prob(tmp_geo)
            i = 0
            tmp = log_prob[i,:]
            #tmp = tmp / sum(tmp)

            if uid in pre_X.keys():
                pre_X[uid] += tmp
            else:
                pre_X[uid] = tmp

            pre_X[lid] = tmp

        fin.close()

    read_node_label(label_file)
    
    
#     filename = 'Foursquare/split_data/'+str(clf_ratio)+'_GMM_embed.txt'

#     fout = open(filename, 'w')
#     for i in range(len(pre_X)):
#         fout.write("{}\n".format(' '.join([str(j) for j in pre_X[id_text[i]] / sum(pre_X[id_text[i]]) ]) ))

#     fout.close()    
    
    #filename = 'Foursquare/split_data/'+str(clf_ratio)+'_GMM_embed.txt'
    filename = 'Gowalla/split_data/'+str(clf_ratio)+'_GMM_embed.txt'
    fout = open(filename, 'w')
    for name, vec in pre_X.items(): 
        fout.write("{} {}\n".format(text_id[name], ' '.join([str(j) for j in pre_X[name] / sum(pre_X[name]) ]) ))

    fout.close()   
    
    #filename = 'Foursquare/split_data/'+str(clf_ratio)+'_GMM_tadw_embed.txt'
    filename = 'Gowalla/split_data/'+str(clf_ratio)+'_GMM_tadw_embed.txt'

    fout = open(filename, 'w')
    for name, vec in pre_X.items(): 
        fout.write("{} {}\n".format(text_id[name], ' '.join([str(j) for j in pre_X[name] / sum(pre_X[name]) ]) ))

    fout.close()    
    
    import random
    import numpy as np
    import os

    node2vec = {}
    random.seed(0)

#     f = open('Foursquare/split_data/'+str(clf_ratio)+'_GMM_embed.txt', 'rb')
#     for i, j in enumerate(f):
#         if j.decode() != '\n':
#             node2vec[i] = list(map(float, j.strip().decode().split(' ')))
            
    #f = open('Foursquare/split_data/'+str(clf_ratio)+'_GMM_embed.txt', 'rb')
    f = open('Gowalla/split_data/'+str(clf_ratio)+'_GMM_embed.txt', 'rb')
    for i, j in enumerate(f):
        if j.decode() != '\n':
            tmp = list(map(float, j.strip().decode().split(' ')))
            node2vec[tmp[0]] = tmp[1:]       

    #f1 = open(os.path.join('Foursquare/split_data/'+str(clf_ratio)+'_reorder_test.txt'), 'rb')
    f1 = open(os.path.join('Gowalla/split_data/'+str(clf_ratio)+'_reorder_test.txt'), 'rb')
    edges = [list(map(int, i.strip().decode().split('\t'))) for i in f1]
    nodes = list(set([i for j in edges for i in j]))
    a = 0
    b = 0
    for i, j in edges:
        if i in node2vec.keys() and j in node2vec.keys():
            dot1 = np.dot(node2vec[i], node2vec[j])
            random_node = random.sample(nodes, 1)[0]
            while random_node == j or random_node not in node2vec.keys():
                random_node = random.sample(nodes, 1)[0]
            dot2 = np.dot(node2vec[i], node2vec[random_node])
            if dot1 > dot2:
                a += 1
            elif dot1 == dot2:
                a += 0.5
            b += 1

    print("Auc value:", float(a) / b)
    
    total_auc[str(clf_ratio)] = float(a) / b

Auc value: 0.681388412077553
Auc value: 0.6876290030517532
Auc value: 0.7807770650400327
Auc value: 0.7341998961644148


In [48]:
f1 = open(os.path.join('Gowalla/split_data/'+str(clf_ratio)+'_reorder_test.txt'), 'rb')
edges = [list(map(int, i.strip().decode().split('\t'))) for i in f1]
nodes = list(set([i for j in edges for i in j]))
a = 0
b = 0
for i, j in edges:
    if i in node2vec.keys() and j in node2vec.keys():
        dot1 = np.dot(node2vec[i], node2vec[j])
        random_node = random.sample(nodes, 1)[0]
        while random_node == j or random_node not in node2vec.keys():
            random_node = random.sample(nodes, 1)[0]
        dot2 = np.dot(node2vec[i], node2vec[random_node])
        if dot1 > dot2:
            a += 1
        elif dot1 == dot2:
            a += 0.5
        b += 1

print("Auc value:", float(a) / b)

total_auc[str(clf_ratio)] = float(a) / b

Auc value: 0.6817471930663815


In [38]:
# Gowalla
print(total_auc)

{'0.1': 0.6813870050932831, '0.2': 0.6876290030517532, '0.3': 0.7807770650400327, '0.4': 0.7341998961644148, '0.5': 0.7512814798306218, '0.6': 0.7363304250927556, '0.7': 0.7855002237098694, '0.8': 0.7845411606792366, '0.9': 0.7610798764118928}


In [36]:
# Forsquare
print(total_auc)

{'0.1': 0.6436106881589944, '0.2': 0.6645565952075834, '0.3': 0.6585232123406636, '0.4': 0.6882582750182459, '0.5': 0.6595915675809343, '0.6': 0.6753516047601875, '0.7': 0.6673879072003847, '0.8': 0.6911802586162485, '0.9': 0.6865694709185514}


In [4]:
for i, name in pre_X.items():
    print(i, name)
    print(name.shape)
    a = 1/0

USER_2907 [-1.10348289e+05 -1.49512726e+03 -9.98518845e+05 -4.34089572e+05
 -8.10118000e+03 -1.74126093e+04 -6.74777675e+04 -3.64420831e+03
 -1.11524463e+04 -4.76844476e+04 -6.02280880e+05 -3.82133154e+05
 -2.71984179e+04 -8.76651166e+03 -3.50872351e+04 -2.32884247e+03
 -7.82167460e+03 -1.24957045e+04 -1.26955356e+04 -2.29969258e+05
 -1.09980843e+05 -2.18606856e+05 -2.81590795e+04 -1.15989521e+02
 -9.60512046e+01 -1.53903440e+03 -1.06552847e+02 -1.35196524e+04
 -3.85325618e+04 -1.98435289e+04 -9.51691371e+07 -1.10298039e+03
 -6.89844027e+04 -2.87149288e+03 -3.68110976e+03 -1.37092452e+04
 -8.60679509e+04 -2.45569074e+04 -9.66215725e+04 -5.02427114e+04
 -3.27360650e+06 -2.52244549e+04 -2.27903192e+04 -2.60971371e+02
 -8.68607753e+03 -1.13838963e+06 -9.65288313e+01 -1.09887612e+05
 -8.93229470e+02 -2.45741715e+07 -2.09448421e+08 -3.83247189e+01
 -1.61819014e+04 -4.69965733e+05 -1.14329919e+04 -1.50565976e+06
 -2.26472947e+07 -5.91070433e+05 -9.57914086e+05 -1.84587296e+03
 -1.12146766e+0

ZeroDivisionError: division by zero

In [46]:
save_file = 'Gowalla/split_data/'+str(clf_ratio)+'_test_file.txt'
fin = open(save_file, 'r')
filename = 'Gowalla/split_data/'+str(clf_ratio)+'_reorder_test.txt'
fout = open(filename, 'w')

while 1:
    l = fin.readline()
    if l == '':
        break
    vec = l.strip().split('\t') 
    uid = text_id[vec[0]]
    lid = text_id[vec[1]]
    fout.write("{}\t{}\n".format(uid,lid))
fin.close()
fout.close()

In [42]:
text_id['USER_93']

1000

In [41]:
id_text[1000]

'USER_93'

In [16]:
user_geo

{'USER_2907': '37.80458895,-122.2945486667',
 'USER_1289': '37.7869452147,-122.4082732201',
 'USER_4821': '33.9956710095,-117.9277983203',
 'USER_5332': '32.7318769977,-117.2019767761',
 'USER_4051': '37.9685607583,-122.0619142056',
 'USER_159': '37.7852494069,-122.4310612679',
 'USER_3908': '37.7572799255,-122.2513532639',
 'USER_5147': '33.5248989333,-117.1532591',
 'USER_3702': '37.8006505833,-122.4108695833',
 'USER_1778': '33.9155627333,-117.8863023833',
 'USER_10055': '37.340068317,-121.90517405',
 'USER_9450': '38.6473114833,-121.35179735',
 'USER_9423': '33.6796225,-117.8622586',
 'USER_424': '33.02128405,-117.2820180167',
 'USER_5136': '33.6134173167,-117.7088274',
 'USER_9583': '37.32653375,-122.00995025',
 'USER_8300': '37.798704571,-122.407296896',
 'USER_7019': '32.695010441,-117.1660351753',
 'USER_1170': '37.687520809,-121.04964417',
 'USER_1407': '33.016340356,-117.1109525988',
 'USER_2885': '34.0718114333,-118.35800885',
 'USER_6591': '37.7730515167,-122.3968063333',
 

In [19]:
pre_X['USER_2588']

array([-2.96663929e+05, -1.41031242e+06, -4.16097322e+05, -2.53809305e+05,
       -7.53245030e+05, -3.03933520e+05, -3.34402021e+04, -7.53140076e+03,
       -5.63408487e+04, -1.69575087e+05, -4.38215395e+05, -1.27882330e+05,
       -1.52593190e+07, -3.74946444e+05, -1.78729924e+04, -5.60989285e+04,
       -6.88868472e+03, -1.30801188e+07, -7.45950670e+02, -7.04984573e+05,
       -1.99214217e+04, -9.52542396e+04, -1.95062480e+06, -7.67317067e+04,
       -5.34846069e+04, -2.72323702e+04, -2.20921213e+05, -9.31327703e+03,
       -6.30677264e+05, -8.76840981e+03, -3.37816509e+07, -2.93496105e+05,
       -5.42607332e+03, -4.92268829e+06, -3.48437780e+06, -2.12252863e+02,
       -1.37608202e+04, -4.05976592e+05, -5.57684838e+04, -1.36988043e+05,
       -9.55574470e+05, -9.42875698e+05, -3.10506451e+04, -5.56755752e+05,
       -5.90243812e+06, -1.59885941e+07, -2.56632585e+04, -1.73856840e+04,
       -6.76489317e+05, -1.46892456e+07, -8.90473940e+07, -3.29067776e+03,
       -1.03478808e+04, -